环境激活过程:

先创建好 conda 环境 (假设名字为 ot), 并安装 conda install ipykernel, 然后 conda deactivate 退出，再运行 python -m ipykernel install --user --name ot --display-name "ot".

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import ot
import sys
import anndata

In [6]:
citeseq = anndata.read_h5ad("./data/citeseq_processed-001.h5ad")

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [17]:
print(citeseq.var['feature_types'].unique())

['GEX', 'ADT']
Categories (2, object): ['ADT', 'GEX']


In [ ]:
X = citeseq.X
print("full data")
print("Matrix Shape:", X.shape)
print("Number of Non-zero Elements:", X.nnz)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())
print("Mean Value without zeros:", X[X != 0].mean())

In [15]:
X = citeseq.X
X = X[:, :13953]
print("GEX data")
print("Matrix Shape:", X.shape)
print("Number of Non-zero Elements:", X.nnz)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())
print("Mean Value without zeros:", X[X != 0].mean())

GEX data
Matrix Shape: (90261, 13953)
Number of Non-zero Elements: 131353640
Density: 0.10429761495639489
Minimum Value: 0.0
Maximum Value: 21078940.0
Mean Value without zeros: 14.417383


In [16]:
X = citeseq.X
X = X[:, -134:]
print("ADT data")
print("Matrix Shape:", X.shape)
print("Number of Non-zero Elements:", X.nnz)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())
print("Mean Value without zeros:", X[X != 0].mean())

ADT data
Matrix Shape: (90261, 134)
Number of Non-zero Elements: 10141612
Density: 0.8384980405910752
Minimum Value: 0.0
Maximum Value: 7.910906
Mean Value without zeros: 0.8842398


In [23]:
X = citeseq.X.toarray()
X = np.log1p(X)
print(np.sum(X > 200) / np.sum(X > 0))
print("Matrix Shape:", X.shape)
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())
print("Mean Value without zeros:", X[X != 0].mean())

0.0
Matrix Shape: (90261, 14087)
Minimum Value: 0.0
Maximum Value: 16.863785
Mean Value without zeros: 1.0193596


In [ ]:
cmap = plt.get_cmap('coolwarm')

plt.imshow(X[:90261 // 2, :14087 // 2], cmap=cmap, aspect='auto')
plt.colorbar()
plt.show()

plt.imshow(X[90261 // 2:, :14087 // 2], cmap=cmap, aspect='auto')
plt.colorbar()
plt.show()

plt.imshow(X[:90261 // 2, 14087 // 2:], cmap=cmap, aspect='auto')
plt.colorbar()
plt.show()

plt.imshow(X[90261 // 2:, 14087 // 2:], cmap=cmap, aspect='auto')
plt.colorbar()
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
cmap = ListedColormap(['blue', 'red'])  # 等于0的数据用蓝色，大于0的数据用红色

plt.imshow(X[:90261 // 2, :14087 // 2], cmap=cmap, aspect='auto')
plt.colorbar()
plt.show()

plt.imshow(X[90261 // 2:, :14087 // 2], cmap=cmap, aspect='auto')
plt.colorbar()
plt.show()

plt.imshow(X[:90261 // 2, 14087 // 2:], cmap=cmap, aspect='auto')
plt.colorbar()
plt.show()

plt.imshow(X[90261 // 2:, 14087 // 2:], cmap=cmap, aspect='auto')
plt.colorbar()
plt.show()